# Bronze Layer Tables

In [0]:
-- USE SCHEMA strata_lab;

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE books_bronze
COMMENT "Raw books data, CDC feed"
AS SELECT * FROM cloud_files("/Volumes/workspace/strata_lab/entrenamiento/books-cdc/", "json",
map("cloudFiles.schemaLocation", "/Volumes/workspace/strata_lab/entrenamiento/books-cdc/schema")
)

# Silver Layer Tables

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE books_silver;

APPLY CHANGES INTO LIVE.books_silver
FROM STREAM(LIVE.books_bronze)
KEYS (book_id)
APPLY AS DELETE WHEN row_status = "DELETE"
SEQUENCE BY row_time
COLUMNS * EXCEPT (row_status, row_time);

# Gold Layer Table

In [0]:
CREATE LIVE TABLE author_counts_state
COMMENT "Number of books per author" AS
SELECT
  author,
  COUNT(*) AS book_count,
  current_timestamp() AS update_time
FROM LIVE.books_silver
GROUP BY author

# DLT Views

In [0]:
-- View is created as normal view instead of live view
-- Cant create live view in free edition
CREATE OR REPLACE VIEW books_sales AS 
SELECT
  b.title,
  o.quantity
  FROM (
    SELECT
      *,
      explode(books.book_id) as book
      FROM orders_cleaned
  ) AS o
  INNER JOIN books_silver AS b
  ON o.book = b.book_id